In [1]:
import pandas as pd

function name: pre_cleansing

Description:

The function cleans the match statistics data and removes the ones that have low to no information for a given match。

Input:

1. Match_stats [DataFrame]: It includes all the players' information for all matches of a given year.

Output:

1. Match_stats_sanitized [DataFrame]: It includes the cleaned data.

In [2]:
phs_2019 = pd.read_csv('phs_2019_playoffs.csv')
phs_2019.head(10)
def pre_cleansing(Match_stats):
    # Remove unused columns
    Match_stats.drop(columns=['start_time','stage','map_type'],inplace=True)

    stat_copy = Match_stats.copy(deep=False)
    hero = Match_stats.iat[0,5]
    flag = False

    # Delete range
    range_begin = 0
    range_end = 0

    for index, row in Match_stats.iterrows():
        # If Alive Time < 50 sec
        if(row.stat_name == 'Time Alive'):
            if(row.stat_amount < 50):
                flag = True
        
        if(row.hero!= hero):  
            hero = row.hero
            range_end = index-1

        # If stat less than 6
            if((range_end-range_begin+1)< 3): 
                flag = True

            if(flag):
                stat_copy.drop(index = range(range_begin,range_end+1),inplace = True)
            range_begin = index
            flag = False
    # Remove All Heroes data
    stat_copy.drop(stat_copy[(stat_copy.hero == "All Heroes")].index, inplace=True) 
    return stat_copy


In [3]:
clean_df_2019 = pre_cleansing(phs_2019) 
clean_df_2019

,match_id,map_name,player,team,stat_name,hero,stat_amount
28,30172,Ilios,Elsa,Chengdu Hunters,All Damage Done,D.Va,13934.906430
29,30172,Ilios,Elsa,Chengdu Hunters,Assists,D.Va,15.000000
30,30172,Ilios,Elsa,Chengdu Hunters,Average Time Alive,D.Va,569.509017
31,30172,Ilios,Elsa,Chengdu Hunters,Barrier Damage Done,D.Va,6703.071256
32,30172,Ilios,Elsa,Chengdu Hunters,Critical Hit Accuracy,D.Va,0.092603
...,...,...,...,...,...,...,...
103967,30157,Watchpoint: Gibraltar,Twilight,Vancouver Titans,Ultimates Used,Ana,6.000000
103968,30157,Watchpoint: Gibraltar,Twilight,Vancouver Titans,Unscoped Accuracy,Ana,0.528777
103969,30157,Watchpoint: Gibraltar,Twilight,Vancouver Titans,Unscoped Hits,Ana,147.000000
103970,30157,Watchpoint: Gibraltar,Twilight,Vancouver Titans,Unscoped Shots,Ana,278.000000


In [13]:
# number of different match_id
print('number of different match_id:', clean_df_2019.match_id.nunique())

number of different match_id: 18


In [9]:
# number of records in each match_id
clean_df_2019.match_id.value_counts()

30175    6265
30163    4555
30154    4415
30173    4405
30156    4133
30151    4073
30176    4040
30172    4032
30152    3803
30153    3409
30155    3292
30162    3011
30164    2991
30159    2799
30157    2672
30160    2528
30161    2383
30158    2382
Name: match_id, dtype: int64

In [4]:
match_result = pd.read_csv('annual_match_result_table.csv')
match_result

,match_year,match_id,map_name,map_winner,winning_team_final_map_score,winning_team_win_rate,map_loser,losing_team_final_map_score,losing_team_win_rate
0,2018,10223,Dorado,Los Angeles Valiant,3,0.600000,San Francisco Shock,2,0.400000
1,2018,10223,Temple of Anubis,Los Angeles Valiant,4,0.571429,San Francisco Shock,3,0.428571
2,2018,10223,Ilios,Los Angeles Valiant,2,0.666667,San Francisco Shock,1,0.333333
3,2018,10223,Numbani,Los Angeles Valiant,2,0.666667,San Francisco Shock,1,0.333333
4,2018,10224,Dorado,Los Angeles Gladiators,1,1.000000,Shanghai Dragons,0,0.000000
...,...,...,...,...,...,...,...,...,...
4511,2021,37442,Dorado,Atlanta Reign,3,0.600000,Dallas Fuel,2,0.400000
4512,2021,37441,Ilios,Shanghai Dragons,2,1.000000,Atlanta Reign,0,0.000000
4513,2021,37441,Hanamura,Shanghai Dragons,3,0.600000,Atlanta Reign,2,0.400000
4514,2021,37441,King's Row,Shanghai Dragons,6,0.545455,Atlanta Reign,5,0.454545


In [45]:
def clean_match(hero_name, match_info):
    clean_match_df = match_info.merge(match_result.iloc[:, :6], how = 'inner', left_on =['match_id', 'map_name', 'team'], right_on =['match_id', 'map_name', 'map_winner'])
    hero_info = clean_match_df[clean_match_df['hero'] == hero_name]
    if len(hero_info) == 0:
        return 'No record about the hero name: ' + hero_name
    else:
        return hero_info
    
Dva_2019 = clean_match('D.Va',clean_df_2019)
Dva_2019

,match_id,map_name,player,team,stat_name,hero,stat_amount,match_year,map_winner,winning_team_final_map_score,winning_team_win_rate
148,30172,Ilios,HOTBA,Guangzhou Charge,All Damage Done,D.Va,2369.906080,2019,Guangzhou Charge,2,0.666667
149,30172,Ilios,HOTBA,Guangzhou Charge,Assists,D.Va,6.000000,2019,Guangzhou Charge,2,0.666667
150,30172,Ilios,HOTBA,Guangzhou Charge,Critical Hit Accuracy,D.Va,0.025758,2019,Guangzhou Charge,2,0.666667
151,30172,Ilios,HOTBA,Guangzhou Charge,Critical Hits,D.Va,34.000000,2019,Guangzhou Charge,2,0.666667
152,30172,Ilios,HOTBA,Guangzhou Charge,Damage - Boosters,D.Va,48.851997,2019,Guangzhou Charge,2,0.666667
...,...,...,...,...,...,...,...,...,...,...,...
11950,30152,Lijiang Tower,Ria,Hangzhou Spark,Time Building Ultimate,D.Va,56.875002,2019,Hangzhou Spark,2,0.666667
11951,30152,Lijiang Tower,Ria,Hangzhou Spark,Time Elapsed per Ultimate Earned,D.Va,106.380884,2019,Hangzhou Spark,2,0.666667
11952,30152,Lijiang Tower,Ria,Hangzhou Spark,Time Played,D.Va,56.456796,2019,Hangzhou Spark,2,0.666667
11953,30152,Lijiang Tower,Ria,Hangzhou Spark,Ultimates Earned - Fractional,D.Va,0.534636,2019,Hangzhou Spark,2,0.666667


In [87]:
Top_11_stat_name = list(clean_df_2019.stat_name.value_counts()[:11].index)
Top_11_stat_name

['Time Played',
 'Time Building Ultimate',
 'Time Alive',
 'Ultimates Earned - Fractional',
 'Time Elapsed per Ultimate Earned',
 'Damage Taken',
 'Healing Received',
 'All Damage Done',
 'Hero Damage Done',
 'Average Time Alive',
 'Barrier Damage Done']

In [88]:
# prepare the data for pivot
Dva_2019_bef_piv = Dva_2019[Dva_2019.stat_name.isin(Top_11_stat_name)].iloc[:, [0,1, 4, 6, 10]]
Dva_2019_bef_piv

,match_id,map_name,stat_name,stat_amount,winning_team_win_rate
148,30172,Ilios,All Damage Done,2369.906080,0.666667
158,30172,Ilios,Damage Taken,2709.025382,0.666667
161,30172,Ilios,Healing Received,2104.717705,0.666667
162,30172,Ilios,Hero Damage Done,2341.693555,0.666667
174,30172,Ilios,Time Alive,170.267005,0.666667
...,...,...,...,...,...
11949,30152,Lijiang Tower,Time Alive,56.875002,0.666667
11950,30152,Lijiang Tower,Time Building Ultimate,56.875002,0.666667
11951,30152,Lijiang Tower,Time Elapsed per Ultimate Earned,106.380884,0.666667
11952,30152,Lijiang Tower,Time Played,56.456796,0.666667


In [89]:
Dva_2019_bef_piv.winning_team_win_rate.value_counts()

0.666667    62
0.750000    21
1.000000    11
0.555556    11
Name: winning_team_win_rate, dtype: int64

In [90]:
Dva_2019_bef_piv.match_id.value_counts()

30175    43
30151    22
30172    19
30173    11
30152    10
Name: match_id, dtype: int64

In [91]:
Dva_2019_bef_piv[Dva_2019_bef_piv['match_id'] == 30151]

,match_id,map_name,stat_name,stat_amount,winning_team_win_rate
8485,30151,Lijiang Tower,All Damage Done,3714.039108,0.666667
8487,30151,Lijiang Tower,Average Time Alive,63.048502,0.666667
8488,30151,Lijiang Tower,Barrier Damage Done,1972.964939,0.666667
8496,30151,Lijiang Tower,Damage Taken,3194.083108,0.666667
8501,30151,Lijiang Tower,Healing Received,1732.191787,0.666667
8502,30151,Lijiang Tower,Hero Damage Done,1627.074170,0.666667
8510,30151,Lijiang Tower,Time Alive,126.097004,0.666667
8511,30151,Lijiang Tower,Time Building Ultimate,140.965004,0.666667
8512,30151,Lijiang Tower,Time Elapsed per Ultimate Earned,80.869725,0.666667
8514,30151,Lijiang Tower,Time Played,146.013389,0.666667


In [92]:
# pivot the data for heat map
#Dva_2019_aft_piv = 
import numpy as np
pd.pivot_table(Dva_2019_bef_piv, values = ['stat_amount'], index = ['match_id', 'map_name','winning_team_win_rate'], columns = ['stat_name'])

                                                                                        

stat_amount  \
stat_name                                            All Damage Done   
match_id map_name              winning_team_win_rate                   
30151    Lijiang Tower         0.666667                  3714.039108   
         Numbani               0.750000                  3701.293239   
30152    Lijiang Tower         0.666667                   635.605500   
30172    Eichenwalde           0.666667                  1989.702972   
         Ilios                 0.666667                  2369.906080   
30173    Lijiang Tower         0.666667                  6153.339690   
30175    Dorado                0.750000                  2885.186710   
         Hanamura              1.000000                  5745.374824   
         Lijiang Tower         0.666667                  2089.249944   
         Watchpoint: Gibraltar 0.555556                 10068.111560   

                                                                         \
stat_name                                            Average Time Alive   
match_id map_name              winning_team_win_rate                      
30151    Lijiang Tower         0.666667                       63.048502   
         Numbani               0.750000                      105.975003   
30152    Lijiang Tower         0.666667                             NaN   
30172    Eichenwalde           0.666667                      180.279006   
         Ilios                 0.666667                             NaN   
30173    Lijiang Tower         0.666667                       60.505802   
30175    Dorado                0.750000                             NaN   
         Hanamura              1.000000                      139.513671   
         Lijiang Tower         0.666667                       77.826002   
         Watchpoint: Gibraltar 0.555556                      224.181507   

                                                                          \
stat_name                                            Barrier Damage Done   
match_id map_name              winning_team_win_rate                       
30151    Lijiang Tower         0.666667                      1972.964939   
         Numbani               0.750000                      1762.193718   
30152    Lijiang Tower         0.666667                        92.266719   
30172    Eichenwalde           0.666667                              NaN   
         Ilios                 0.666667                              NaN   
30173    Lijiang Tower         0.666667                      3054.077865   
30175    Dorado                0.750000                       410.652885   
         Hanamura              1.000000                      3167.141608   
         Lijiang Tower         0.666667                       982.235920   
         Watchpoint: Gibraltar 0.555556                      3780.705496   

                                                                   \
stat_name                                            Damage Taken   
match_id map_name              winning_team_win_rate                
30151    Lijiang Tower         0.666667               3194.083108   
         Numbani               0.750000               3075.597053   
30152    Lijiang Tower         0.666667               1043.024597   
30172    Eichenwalde           0.666667               2318.310134   
         Ilios                 0.666667               2709.025382   
30173    Lijiang Tower         0.666667               6134.634609   
30175    Dorado                0.750000               2096.771551   
         Hanamura              1.000000               6299.791434   
         Lijiang Tower         0.666667               2905.669602   
         Watchpoint: Gibraltar 0.555556               8049.090241   

                                                                       \
stat_name                                            Healing Received   
match_id map_name              winning_team_win_rate                    
30151    Lijiang Tower         0.66666

In [6]:
# test dataset
# clean dataset for phs_2020_1
phs_2020 = pd.read_csv('phs_2020_1.csv')
phs_2020.head(10)
def pre_cleansing(Match_stats):
    # Remove unused columns
    Match_stats.drop(columns=['start_time','stage','map_type'],inplace=True)

    stat_copy = Match_stats.copy(deep=False)
    hero = Match_stats.iat[0,5]
    flag = False

    # Delete range
    range_begin = 0
    range_end = 0

    for index, row in Match_stats.iterrows():
        # If Alive Time < 50 sec
        if(row.stat_name == 'Time Alive'):
            if(row.stat_amount < 50):
                flag = True
        
        if(row.hero!= hero):
            hero = row.hero
            range_end = index-1

        # If stat less than 6
            if((range_end-range_begin+1)< 3): 
                flag = True

            if(flag):
                stat_copy.drop(index = range(range_begin,range_end+1),inplace = True)
            range_begin = index
            flag = False
    # Remove All Heroes data
    stat_copy.drop(stat_copy[(stat_copy.hero == "All Heroes")].index, inplace=True) 
    return stat_copy
clean_df_2020 = pre_cleansing(phs_2020) 
clean_df_2020


,match_id,map_name,player,team,stat_name,hero,stat_amount
22,30991,Lijiang Tower,BenBest,Paris Eternal,All Damage Done,Reinhardt,6674.164055
23,30991,Lijiang Tower,BenBest,Paris Eternal,Assists,Reinhardt,11.000000
24,30991,Lijiang Tower,BenBest,Paris Eternal,Average Time Alive,Reinhardt,60.199502
25,30991,Lijiang Tower,BenBest,Paris Eternal,Barrier Damage Done,Reinhardt,1875.000000
26,30991,Lijiang Tower,BenBest,Paris Eternal,Charge Kills,Reinhardt,1.000000
...,...,...,...,...,...,...,...
760106,34751,Rialto,eqo,Philadelphia Fusion,Time Alive,Tracer,3.675000
760107,34751,Rialto,eqo,Philadelphia Fusion,Time Building Ultimate,Tracer,3.675000
760108,34751,Rialto,eqo,Philadelphia Fusion,Time Elapsed per Ultimate Earned,Tracer,308.700003
760109,34751,Rialto,eqo,Philadelphia Fusion,Time Played,Tracer,3.896239


In [34]:
# Top 11 stat_name in year 2019 
clean_df_2019.stat_name.value_counts()[:11]

Time Played                         1703
Time Building Ultimate              1703
Time Alive                          1703
Ultimates Earned - Fractional       1701
Time Elapsed per Ultimate Earned    1701
Damage Taken                        1679
Healing Received                    1677
All Damage Done                     1658
Hero Damage Done                    1656
Average Time Alive                  1640
Barrier Damage Done                 1612
Name: stat_name, dtype: int64

In [35]:
# Top 11 stat_name in year 2020
clean_df_2020.stat_name.value_counts()[:11]

Time Alive                          12351
Time Building Ultimate              12351
Time Played                         12351
Time Elapsed per Ultimate Earned    12339
Ultimates Earned - Fractional       12339
Damage Taken                        12135
Healing Received                    12104
All Damage Done                     12084
Hero Damage Done                    12076
Barrier Damage Done                 11909
Average Time Alive                  11852
Name: stat_name, dtype: int64